In [1]:
import spacy
import pandas as pd
import joblib
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder,StandardScaler
import re
import pickle

In [2]:
from xgboost import XGBClassifier
df_film = pd.read_csv('../data/data-film-final.csv')
intent_model = XGBClassifier()
intent_model.load_model('../models/intent_recognition_model.json')
ner_model = spacy.load('../models/model_ner_NVEB')
with open('../models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)
with open('../models/scaler.pkl', 'rb') as f:
     scaler = pickle.load(f)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
def get_embedding(text):
     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
     with torch.no_grad():
          outputs = model(**inputs)
          embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
     return normalize(embedding, norm="l2")  # chuẩn hóa nếu muốn


In [6]:
input = 'gợi ý phim hành động 2025 có rating cao'
def predict_intent(text, model, tokenizer, scaler, intent_model, label_encoder):
    text = re.sub(r"[^a-zA-Z0-9À-ỹà-ỹ\s]", "", text.lower())
    text = re.sub(r"\s+", " ", text).strip()

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)

    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embedding = normalize(embedding, norm="l2")

    input_scaled = scaler.transform(embedding)
    pred = intent_model.predict(input_scaled)[0]
    intent_label = label_encoder.inverse_transform([pred])[0]

    return intent_label

print(predict_intent(input, model, tokenizer, scaler, intent_model, label_encoder))

goi_y_theo_nam


In [16]:
def add_year_prefix(text):
    # Tìm tất cả các số có 4 chữ số (có thể là năm)
    text = text.strip().lower()
    text = re.sub(r'(\b\d{4}\b)', r'năm năm \1', text)
    return text
input2 = 'gợi ý phim hành động 2025 có rating cao'

input2 = add_year_prefix(input2)
print(input2)
doc = ner_model(input2)
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")
len(doc.ents)

gợi ý phim hành động năm năm 2025 có rating cao
Entity: hành động, Label: genre
Entity: 2025, Label: year
Entity: cao, Label: rating_level


3